In [1]:
from game.game import Game
from game.data.generator import create_numbers
import torch
import os

def load(last_save, n, vocab, sender, receiver, embedding):
    epochs = last_save + 1
    save_name = "game/models/game10_14_{}_{}_{}_{}_{}".format(n, vocab, sender, receiver, embedding)
    params = [
        '--batch_size=1',
        '--n_epochs={}'.format(epochs),
        '--vocab_size={}'.format(vocab),
        '--sender_hidden={}'.format(sender),
        '--receiver_hidden={}'.format(receiver),
        '--lr=0.001',
        '--max_len=1',
        '--sender_entropy=0.5',
        '--sender_embedding={}'.format(embedding),
        '--receiver_embedding={}'.format(embedding),
        '--validation_freq=10',
        '--train_data=game/data/train10.txt',
        '--validation_data=game/data/validation10.txt',
        '--load_from_checkpoint={}_{}.tar'.format(save_name, last_save)
    ]
    game10_14 = Game(params)
    game10_14.play()
    game10_14.save_checkpoint(epochs, save_name)
    return game10_14

In [5]:
extra_tests = {}
def test_above_limit(model, start, end, number_of_samples):
    # creates more samples for model to predict
    inputs, labels = extra_tests.get((start, end, number_of_samples), (None, None))
    if(inputs is None or labels is None):
        inputs, labels = create_numbers(end, number_of_samples, False, False, start)
        extra_tests[(start, end, number_of_samples)] = (inputs, labels)
        print("Inputs: {}".format(inputs))
        print("Labels: {}".format(labels))
    inputs = torch.FloatTensor(inputs)
    labels = torch.FloatTensor(labels)
    prediction = model.game.forward(inputs,labels)
    #print(prediction)
    messages = prediction[1].message.tolist()
    outputs = prediction[1].receiver_output.tolist()
    for i in range(len(outputs)):
        print("{} {} {} {}".format(inputs[i].tolist(), messages[i], outputs[i][0], labels[i].tolist()))
    print("Testing {} samples of numbers that sum up to [{} - {}]".format(number_of_samples, start, end))
    print("Unique outputs: {}".format(set([item[0] for item in outputs])))
    print("Accuracy: {}".format(prediction[1].aux['acc'].mean().item()))

In [58]:
def test_all_set(game, n):
    inputs = []
    labels = []
    for i in range(n+1):
        for j in range(n+1):
            if i+j <= n:
                inputs.append([i, j])
                labels.append([i+j]) 
    inputs = torch.FloatTensor(inputs)
    labels = torch.FloatTensor(labels)
    prediction = game.game.forward(inputs,labels)
    messages = prediction[1].message.tolist()
    outputs = prediction[1].receiver_output.tolist()
    print("Testing {} samples of numbers that sum up to {}".format(len(inputs), n))
    for i in range(len(outputs)):
        print("{} {} {} {}".format(inputs[i].tolist(), messages[i], outputs[i][0], labels[i].tolist()))
    print("Unique outputs: {}".format(set([item[0] for item in outputs])))
    print("Accuracy: {}".format(prediction[1].aux['acc'].mean().item()))

In [95]:
def test_some_other_numbers(game):
    inputs = torch.FloatTensor([[6.0, 4.0],[6.0, 4.0],[9.0, 1.0],[9.0, 1.0],[9.0, 1.0],[1.0, 1.0],[1.0, 1.0],[1.0, 1.0], [0.0, 1.0], [0.0, 0.0], [-1.0, 4.0], [5.0, -2.0], [-1.0, 0.0], [-2.0, -2.0], [-9.0, -1.0], [8.0, -3.0], [-10.0, -10.0]])
    labels = torch.FloatTensor([[10.],[10.],[10.],[10.],[10.],[2.0],[2.0],[2.0],[1.0],[0.0],[3.0],[3.0], [-1.0], [-4.0], [-10.0], [5.0], [-20.0]])
    prediction = game.game.forward(inputs,labels)
    messages = prediction[1].message.tolist()
    outputs = prediction[1].receiver_output.tolist()
    for i in range(len(outputs)):
        print("{} {} {} {}".format(inputs[i].tolist(), messages[i], outputs[i][0], labels[i].tolist()))
    print("Unique outputs: {}".format(set([item[0] for item in outputs])))
    print("Accuracy: {}".format(prediction[1].aux['acc'].mean().item()))

In [3]:
game = load(1500, 1, 32, 64, 32, 7) # game10_14_1_32_64_32_7_1500

Preparing dataset...
Samples in training set: 52
Samples in test set: 14

----------
Building model...
# Initializing model, trainer, and optimizer from game/models/game10_14_1_32_64_32_7_1500.tar
# loading trainer state from game/models/game10_14_1_32_64_32_7_1500.tar
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=64, bias=True)
    )
    (hidden_to_output): Linear(in_features=64, out_features=32, bias=True)
    (embedding): Embedding(32, 7)
    (cells): ModuleList(
      (0): RNNCell(7, 64)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=32, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(7, 32, batch_first=True)
      (embedding): Embedding(32, 7)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
Number of parameters 8744

----------
Start of training... 2021-11-10 19:56:17.008019
{"loss": -0.383

In [10]:
test_above_limit(game, 0, 10, None)

Generating 66 samples...
Inputs: [[0, 0], [9, 1], [7, 1], [0, 5], [3, 7], [1, 2], [3, 3], [1, 0], [0, 1], [3, 4], [4, 6], [2, 4], [6, 4], [1, 1], [2, 0], [1, 3], [5, 3], [3, 0], [5, 0], [0, 2], [6, 2], [5, 1], [0, 4], [2, 5], [0, 3], [2, 3], [7, 3], [3, 2], [5, 4], [4, 1], [8, 2], [10, 0], [1, 7], [3, 5], [3, 6], [2, 2], [5, 2], [7, 0], [7, 2], [4, 0], [0, 10], [1, 4], [4, 2], [2, 1], [3, 1], [6, 1], [0, 7], [1, 8], [0, 6], [0, 9], [2, 6], [6, 0], [8, 0], [1, 9], [6, 3], [1, 6], [4, 3], [0, 8], [9, 0], [1, 5], [8, 1], [5, 5], [4, 4], [4, 5], [2, 7], [2, 8]]
Labels: [[0], [10], [8], [5], [10], [3], [6], [1], [1], [7], [10], [6], [10], [2], [2], [4], [8], [3], [5], [2], [8], [6], [4], [7], [3], [5], [10], [5], [9], [5], [10], [10], [8], [8], [9], [4], [7], [7], [9], [4], [10], [5], [6], [3], [4], [7], [7], [9], [6], [9], [8], [6], [8], [10], [9], [7], [7], [8], [9], [6], [9], [10], [8], [9], [9], [10]]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[9.0, 1.0] [13, 0] 9.9131498336792 [10.0]
[

In [12]:
test_above_limit(game, 11, 12, 20)

Generating 20 samples...
Inputs: [[11, 1], [2, 9], [6, 5], [0, 11], [1, 11], [12, 0], [10, 2], [9, 3], [10, 1], [9, 2], [7, 4], [7, 5], [11, 0], [5, 6], [0, 12], [8, 4], [1, 10], [6, 6], [5, 7], [3, 9]]
Labels: [[12], [11], [11], [11], [12], [12], [12], [12], [11], [11], [11], [12], [11], [11], [12], [12], [11], [12], [12], [12]]
[11.0, 1.0] [13, 0] 9.9131498336792 [12.0]
[2.0, 9.0] [1, 0] 9.226584434509277 [11.0]
[6.0, 5.0] [13, 0] 9.9131498336792 [11.0]
[0.0, 11.0] [28, 0] 9.217903137207031 [11.0]
[1.0, 11.0] [13, 0] 9.9131498336792 [12.0]
[12.0, 0.0] [1, 0] 9.226584434509277 [12.0]
[10.0, 2.0] [13, 0] 9.9131498336792 [12.0]
[9.0, 3.0] [1, 0] 9.226584434509277 [12.0]
[10.0, 1.0] [13, 0] 9.9131498336792 [11.0]
[9.0, 2.0] [13, 0] 9.9131498336792 [11.0]
[7.0, 4.0] [13, 0] 9.9131498336792 [11.0]
[7.0, 5.0] [13, 0] 9.9131498336792 [12.0]
[11.0, 0.0] [1, 0] 9.226584434509277 [11.0]
[5.0, 6.0] [13, 0] 9.9131498336792 [11.0]
[0.0, 12.0] [17, 0] 9.22646427154541 [12.0]
[8.0, 4.0] [17, 0] 9.22

In [13]:
test_above_limit(game, 50, 51, 20)

Generating 20 samples...
Inputs: [[33, 17], [21, 30], [36, 15], [37, 14], [4, 47], [11, 39], [31, 20], [16, 34], [20, 30], [7, 43], [22, 28], [44, 6], [11, 40], [25, 25], [14, 36], [23, 28], [6, 44], [45, 5], [5, 45], [16, 35]]
Labels: [[50], [51], [51], [51], [51], [50], [51], [50], [50], [50], [50], [50], [51], [50], [50], [51], [50], [50], [50], [51]]
[33.0, 17.0] [13, 0] 9.9131498336792 [50.0]
[21.0, 30.0] [13, 0] 9.9131498336792 [51.0]
[36.0, 15.0] [13, 0] 9.9131498336792 [51.0]
[37.0, 14.0] [13, 0] 9.9131498336792 [51.0]
[4.0, 47.0] [13, 0] 9.9131498336792 [51.0]
[11.0, 39.0] [13, 0] 9.9131498336792 [50.0]
[31.0, 20.0] [13, 0] 9.9131498336792 [51.0]
[16.0, 34.0] [13, 0] 9.9131498336792 [50.0]
[20.0, 30.0] [13, 0] 9.9131498336792 [50.0]
[7.0, 43.0] [13, 0] 9.9131498336792 [50.0]
[22.0, 28.0] [13, 0] 9.9131498336792 [50.0]
[44.0, 6.0] [13, 0] 9.9131498336792 [50.0]
[11.0, 40.0] [13, 0] 9.9131498336792 [51.0]
[25.0, 25.0] [13, 0] 9.9131498336792 [50.0]
[14.0, 36.0] [13, 0] 9.9131498

In [14]:
test_above_limit(game, 1000, 1001, 20)

Generating 20 samples...
Inputs: [[268, 733], [701, 299], [614, 387], [113, 888], [372, 629], [993, 7], [397, 604], [46, 954], [482, 518], [902, 98], [1, 999], [444, 557], [34, 966], [17, 983], [952, 49], [364, 637], [504, 496], [250, 751], [584, 417], [615, 385]]
Labels: [[1001], [1000], [1001], [1001], [1001], [1000], [1001], [1000], [1000], [1000], [1000], [1001], [1000], [1000], [1001], [1001], [1000], [1001], [1001], [1000]]
[268.0, 733.0] [13, 0] 9.9131498336792 [1001.0]
[701.0, 299.0] [13, 0] 9.9131498336792 [1000.0]
[614.0, 387.0] [13, 0] 9.9131498336792 [1001.0]
[113.0, 888.0] [13, 0] 9.9131498336792 [1001.0]
[372.0, 629.0] [13, 0] 9.9131498336792 [1001.0]
[993.0, 7.0] [13, 0] 9.9131498336792 [1000.0]
[397.0, 604.0] [29, 0] 9.241236686706543 [1001.0]
[46.0, 954.0] [13, 0] 9.9131498336792 [1000.0]
[482.0, 518.0] [13, 0] 9.9131498336792 [1000.0]
[902.0, 98.0] [13, 0] 9.9131498336792 [1000.0]
[1.0, 999.0] [13, 0] 9.9131498336792 [1000.0]
[444.0, 557.0] [13, 0] 9.9131498336792 [10

In [16]:
test_above_limit(game, 1000000, 1000001, 20)

Generating 20 samples...
Inputs: [[563817, 436183], [444942, 555058], [103231, 896770], [881040, 118961], [134532, 865469], [122241, 877760], [30956, 969044], [852855, 147145], [938309, 61692], [879280, 120720], [374403, 625597], [920007, 79993], [974718, 25282], [577601, 422400], [996782, 3218], [944406, 55595], [99109, 900892], [447832, 552168], [235413, 764587], [329036, 670965]]
Labels: [[1000000], [1000000], [1000001], [1000001], [1000001], [1000001], [1000000], [1000000], [1000001], [1000000], [1000000], [1000000], [1000000], [1000001], [1000000], [1000001], [1000001], [1000000], [1000000], [1000001]]
[563817.0, 436183.0] [13, 0] 9.9131498336792 [1000000.0]
[444942.0, 555058.0] [13, 0] 9.9131498336792 [1000000.0]
[103231.0, 896770.0] [13, 0] 9.9131498336792 [1000001.0]
[881040.0, 118961.0] [13, 0] 9.9131498336792 [1000001.0]
[134532.0, 865469.0] [13, 0] 9.9131498336792 [1000001.0]
[122241.0, 877760.0] [13, 0] 9.9131498336792 [1000001.0]
[30956.0, 969044.0] [13, 0] 9.9131498336792

In [94]:
test_some_other_numbers(game)

NameError: name 'test_some_other_numbers' is not defined

[-1.0, 4.0] [30, 0] 3.0951414108276367 [3.0]
[5.0, -2.0] [30, 0] 3.0951414108276367 [3.0]
[-1.0, 0.0] [21, 0] 1.217471957206726 [-1.0]
[-2.0, -2.0] [21, 0] 1.217471957206726 [-4.0]
[-9.0, -1.0] [21, 0] 1.217471957206726 [-10.0]
[8.0, -3.0] [9, 0] 5.03515625 [5.0]
[-10.0, -10.0] [21, 0] 1.217471957206726 [-20.0]
Unique outputs: {1.217471957206726, 3.0951414108276367, 5.03515625}
Accuracy: 0.4285714328289032


In [31]:
game2 = load(1500, 1, 64, 128, 64, 8) # game10_14_1_64_128_64_8_1500

Preparing dataset...
Samples in training set: 52
Samples in test set: 14

----------
Building model...
# Initializing model, trainer, and optimizer from game/models/game10_14_1_64_128_64_8_1500.tar
# loading trainer state from game/models/game10_14_1_64_128_64_8_1500.tar
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=128, bias=True)
    )
    (hidden_to_output): Linear(in_features=128, out_features=64, bias=True)
    (embedding): Embedding(64, 8)
    (cells): ModuleList(
      (0): RNNCell(8, 128)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=64, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(8, 64, batch_first=True)
      (embedding): Embedding(64, 8)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
Number of parameters 32137

----------
Start of training... 2021-11-10 20:14:25.137922
{"loss": 

In [32]:
test_above_limit(game2, 0, 10, None)

[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[9.0, 1.0] [25, 0] 9.520702362060547 [10.0]
[7.0, 1.0] [40, 0] 7.04573917388916 [8.0]
[0.0, 5.0] [21, 0] 5.972923755645752 [5.0]
[3.0, 7.0] [2, 0] 9.31519889831543 [10.0]
[1.0, 2.0] [53, 0] 2.949716091156006 [3.0]
[3.0, 3.0] [42, 0] 5.884256362915039 [6.0]
[1.0, 0.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[3.0, 4.0] [24, 0] 6.985294818878174 [7.0]
[4.0, 6.0] [26, 0] 9.526752471923828 [10.0]
[2.0, 4.0] [31, 0] 5.9190802574157715 [6.0]
[6.0, 4.0] [26, 0] 9.526752471923828 [10.0]
[1.0, 1.0] [17, 0] 1.9181718826293945 [2.0]
[2.0, 0.0] [17, 0] 1.9181718826293945 [2.0]
[1.0, 3.0] [39, 0] 4.021642208099365 [4.0]
[5.0, 3.0] [13, 0] 7.766280651092529 [8.0]
[3.0, 0.0] [53, 0] 2.949716091156006 [3.0]
[5.0, 0.0] [9, 0] 5.006842613220215 [5.0]
[0.0, 2.0] [53, 0] 2.949716091156006 [2.0]
[6.0, 2.0] [38, 0] 7.888430118560791 [8.0]
[5.0, 1.0] [42, 0] 5.884256362915039 [6.0]
[0.0, 4.0] [39, 0] 4.021642208099365 [4.0]
[2.0,

In [33]:
test_above_limit(game2, 11, 12, 20)

[11.0, 1.0] [36, 0] 9.50093936920166 [12.0]
[2.0, 9.0] [35, 0] 9.32690143585205 [11.0]
[6.0, 5.0] [63, 0] 9.525580406188965 [11.0]
[0.0, 11.0] [2, 0] 9.31519889831543 [11.0]
[1.0, 11.0] [2, 0] 9.31519889831543 [12.0]
[12.0, 0.0] [18, 0] 9.529882431030273 [12.0]
[10.0, 2.0] [63, 0] 9.525580406188965 [12.0]
[9.0, 3.0] [63, 0] 9.525580406188965 [12.0]
[10.0, 1.0] [2, 0] 9.31519889831543 [11.0]
[9.0, 2.0] [32, 0] 9.501254081726074 [11.0]
[7.0, 4.0] [49, 0] 8.65815544128418 [11.0]
[7.0, 5.0] [41, 0] 9.529383659362793 [12.0]
[11.0, 0.0] [32, 0] 9.501254081726074 [11.0]
[5.0, 6.0] [2, 0] 9.31519889831543 [11.0]
[0.0, 12.0] [36, 0] 9.50093936920166 [12.0]
[8.0, 4.0] [63, 0] 9.525580406188965 [12.0]
[1.0, 10.0] [36, 0] 9.50093936920166 [11.0]
[6.0, 6.0] [36, 0] 9.50093936920166 [12.0]
[5.0, 7.0] [2, 0] 9.31519889831543 [12.0]
[3.0, 9.0] [2, 0] 9.31519889831543 [12.0]
Testing 20 samples of numbers that sum up to [11 - 12]
Unique outputs: {8.65815544128418, 9.50093936920166, 9.31519889831543, 9.5

In [34]:
test_above_limit(game2, 50, 51, 20)

[33.0, 17.0] [36, 0] 9.50093936920166 [50.0]
[21.0, 30.0] [41, 0] 9.529383659362793 [51.0]
[36.0, 15.0] [18, 0] 9.529882431030273 [51.0]
[37.0, 14.0] [35, 0] 9.32690143585205 [51.0]
[4.0, 47.0] [2, 0] 9.31519889831543 [51.0]
[11.0, 39.0] [26, 0] 9.526752471923828 [50.0]
[31.0, 20.0] [36, 0] 9.50093936920166 [51.0]
[16.0, 34.0] [51, 0] 9.50953197479248 [50.0]
[20.0, 30.0] [2, 0] 9.31519889831543 [50.0]
[7.0, 43.0] [2, 0] 9.31519889831543 [50.0]
[22.0, 28.0] [51, 0] 9.50953197479248 [50.0]
[44.0, 6.0] [2, 0] 9.31519889831543 [50.0]
[11.0, 40.0] [35, 0] 9.32690143585205 [51.0]
[25.0, 25.0] [25, 0] 9.520702362060547 [50.0]
[14.0, 36.0] [36, 0] 9.50093936920166 [50.0]
[23.0, 28.0] [2, 0] 9.31519889831543 [51.0]
[6.0, 44.0] [35, 0] 9.32690143585205 [50.0]
[45.0, 5.0] [35, 0] 9.32690143585205 [50.0]
[5.0, 45.0] [2, 0] 9.31519889831543 [50.0]
[16.0, 35.0] [36, 0] 9.50093936920166 [51.0]
Testing 20 samples of numbers that sum up to [50 - 51]
Unique outputs: {9.50093936920166, 9.32690143585205, 

In [35]:
test_above_limit(game2, 1000, 1050, 20)

Generating 20 samples...
Inputs: [[618, 414], [1007, 22], [275, 763], [854, 182], [588, 455], [823, 192], [776, 247], [13, 1022], [801, 234], [347, 660], [459, 580], [177, 873], [335, 687], [23, 984], [444, 606], [803, 244], [815, 195], [455, 583], [147, 861], [551, 492]]
Labels: [[1032], [1029], [1038], [1036], [1043], [1015], [1023], [1035], [1035], [1007], [1039], [1050], [1022], [1007], [1050], [1047], [1010], [1038], [1008], [1043]]
[618.0, 414.0] [2, 0] 9.31519889831543 [1032.0]
[1007.0, 22.0] [5, 0] 9.519076347351074 [1029.0]
[275.0, 763.0] [2, 0] 9.31519889831543 [1038.0]
[854.0, 182.0] [18, 0] 9.529882431030273 [1036.0]
[588.0, 455.0] [25, 0] 9.520702362060547 [1043.0]
[823.0, 192.0] [63, 0] 9.525580406188965 [1015.0]
[776.0, 247.0] [32, 0] 9.501254081726074 [1023.0]
[13.0, 1022.0] [18, 0] 9.529882431030273 [1035.0]
[801.0, 234.0] [41, 0] 9.529383659362793 [1035.0]
[347.0, 660.0] [2, 0] 9.31519889831543 [1007.0]
[459.0, 580.0] [18, 0] 9.529882431030273 [1039.0]
[177.0, 873.0] 

In [97]:
test_some_other_numbers(game2)

[6.0, 4.0] [36, 0] 9.50093936920166 [10.0]
[6.0, 4.0] [5, 0] 9.519076347351074 [10.0]
[9.0, 1.0] [36, 0] 9.50093936920166 [10.0]
[9.0, 1.0] [41, 0] 9.529383659362793 [10.0]
[9.0, 1.0] [51, 0] 9.50953197479248 [10.0]
[1.0, 1.0] [17, 0] 1.9181718826293945 [2.0]
[1.0, 1.0] [17, 0] 1.9181718826293945 [2.0]
[1.0, 1.0] [53, 0] 2.949716091156006 [2.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[-1.0, 4.0] [53, 0] 2.949716091156006 [3.0]
[5.0, -2.0] [53, 0] 2.949716091156006 [3.0]
[-1.0, 0.0] [14, 0] 0.6579403281211853 [-1.0]
[-2.0, -2.0] [14, 0] 0.6579403281211853 [-4.0]
[-9.0, -1.0] [14, 0] 0.6579403281211853 [-10.0]
[8.0, -3.0] [9, 0] 5.006842613220215 [5.0]
[-10.0, -10.0] [14, 0] 0.6579403281211853 [-20.0]
Unique outputs: {0.6579403281211853, 1.9181718826293945, 2.949716091156006, 5.006842613220215, 9.50093936920166, 9.50953197479248, 9.519076347351074, 9.529383659362793}
Accuracy: 0.6470588445663452


In [61]:
test_all_set(game, 10)

Testing 66 samples of numbers that sum up to 10
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 2.0] [21, 0] 1.217471957206726 [2.0]
[0.0, 3.0] [30, 0] 3.0951414108276367 [3.0]
[0.0, 4.0] [20, 0] 3.9991846084594727 [4.0]
[0.0, 5.0] [9, 0] 5.03515625 [5.0]
[0.0, 6.0] [7, 0] 5.878513813018799 [6.0]
[0.0, 7.0] [4, 0] 6.976547718048096 [7.0]
[0.0, 8.0] [23, 0] 7.966855049133301 [8.0]
[0.0, 9.0] [2, 0] 9.221352577209473 [9.0]
[0.0, 10.0] [13, 0] 9.9131498336792 [10.0]
[1.0, 0.0] [21, 0] 1.217471957206726 [1.0]
[1.0, 1.0] [21, 0] 1.217471957206726 [2.0]
[1.0, 2.0] [30, 0] 3.0951414108276367 [3.0]
[1.0, 3.0] [20, 0] 3.9991846084594727 [4.0]
[1.0, 4.0] [9, 0] 5.03515625 [5.0]
[1.0, 5.0] [18, 0] 5.964083671569824 [6.0]
[1.0, 6.0] [24, 0] 6.954435348510742 [7.0]
[1.0, 7.0] [17, 0] 9.22646427154541 [8.0]
[1.0, 8.0] [17, 0] 9.22646427154541 [9.0]
[1.0, 9.0] [13, 0] 9.9131498336792 [10.0]
[2.0, 0.0] [21, 0] 1.217471957206726 [2.0]
[2.0, 1.0] [30, 0] 3.095

In [68]:
test_all_set(game2, 10)

Testing 66 samples of numbers that sum up to 10
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 2.0] [53, 0] 2.949716091156006 [2.0]
[0.0, 3.0] [53, 0] 2.949716091156006 [3.0]
[0.0, 4.0] [39, 0] 4.021642208099365 [4.0]
[0.0, 5.0] [42, 0] 5.884256362915039 [5.0]
[0.0, 6.0] [42, 0] 5.884256362915039 [6.0]
[0.0, 7.0] [29, 0] 7.1015729904174805 [7.0]
[0.0, 8.0] [52, 0] 7.572460651397705 [8.0]
[0.0, 9.0] [41, 0] 9.529383659362793 [9.0]
[0.0, 10.0] [2, 0] 9.31519889831543 [10.0]
[1.0, 0.0] [14, 0] 0.6579403281211853 [1.0]
[1.0, 1.0] [17, 0] 1.9181718826293945 [2.0]
[1.0, 2.0] [53, 0] 2.949716091156006 [3.0]
[1.0, 3.0] [39, 0] 4.021642208099365 [4.0]
[1.0, 4.0] [9, 0] 5.006842613220215 [5.0]
[1.0, 5.0] [21, 0] 5.972923755645752 [6.0]
[1.0, 6.0] [7, 0] 6.889515399932861 [7.0]
[1.0, 7.0] [1, 0] 8.054494857788086 [8.0]
[1.0, 8.0] [61, 0] 7.875339984893799 [9.0]
[1.0, 9.0] [26, 0] 9.526752471923828 [10.0]
[2.0, 0.0] [17, 0] 1.9181718826293945 [2.0]
[2

In [69]:
game3 = load(1500, 1, 32, 32, 32, 7) #game10_14_1_32_32_32_7_1500.tar

Preparing dataset...
Samples in training set: 52
Samples in test set: 14

----------
Building model...
# Initializing model, trainer, and optimizer from game/models/game10_14_1_32_32_32_7_1500.tar
# loading trainer state from game/models/game10_14_1_32_32_32_7_1500.tar
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=32, bias=True)
    )
    (hidden_to_output): Linear(in_features=32, out_features=32, bias=True)
    (embedding): Embedding(32, 7)
    (cells): ModuleList(
      (0): RNNCell(7, 32)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=32, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(7, 32, batch_first=True)
      (embedding): Embedding(32, 7)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
Number of parameters 4264

----------
Start of training... 2021-11-10 20:44:45.041875
{"loss": -0.893

In [70]:
test_all_set(game3, 10)

Testing 66 samples of numbers that sum up to 10
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 2.0] [1, 0] 2.8394417762756348 [2.0]
[0.0, 3.0] [1, 0] 2.8394417762756348 [3.0]
[0.0, 4.0] [26, 0] 4.2848944664001465 [4.0]
[0.0, 5.0] [13, 0] 5.600855350494385 [5.0]
[0.0, 6.0] [0, 0] 6.816980361938477 [6.0]
[0.0, 7.0] [28, 0] 6.052976131439209 [7.0]
[0.0, 8.0] [19, 0] 8.234979629516602 [8.0]
[0.0, 9.0] [3, 0] 8.957698822021484 [9.0]
[0.0, 10.0] [8, 0] 9.894006729125977 [10.0]
[1.0, 0.0] [25, 0] 0.6592649221420288 [1.0]
[1.0, 1.0] [1, 0] 2.8394417762756348 [2.0]
[1.0, 2.0] [1, 0] 2.8394417762756348 [3.0]
[1.0, 3.0] [26, 0] 4.2848944664001465 [4.0]
[1.0, 4.0] [31, 0] 5.577843189239502 [5.0]
[1.0, 5.0] [28, 0] 6.052976131439209 [6.0]
[1.0, 6.0] [10, 0] 6.957682132720947 [7.0]
[1.0, 7.0] [19, 0] 8.234979629516602 [8.0]
[1.0, 8.0] [6, 0] 9.84208869934082 [9.0]
[1.0, 9.0] [21, 0] 9.655567169189453 [10.0]
[2.0, 0.0] [1, 0] 2.8394417762756348 [2.0]
[2.

In [105]:
test_above_limit(game2, 11, 12, 20)

[11.0, 1.0] [36, 0] 9.50093936920166 [12.0]
[2.0, 9.0] [63, 0] 9.525580406188965 [11.0]
[6.0, 5.0] [2, 0] 9.31519889831543 [11.0]
[0.0, 11.0] [41, 0] 9.529383659362793 [11.0]
[1.0, 11.0] [2, 0] 9.31519889831543 [12.0]
[12.0, 0.0] [36, 0] 9.50093936920166 [12.0]
[10.0, 2.0] [36, 0] 9.50093936920166 [12.0]
[9.0, 3.0] [25, 0] 9.520702362060547 [12.0]
[10.0, 1.0] [18, 0] 9.529882431030273 [11.0]
[9.0, 2.0] [36, 0] 9.50093936920166 [11.0]
[7.0, 4.0] [5, 0] 9.519076347351074 [11.0]
[7.0, 5.0] [36, 0] 9.50093936920166 [12.0]
[11.0, 0.0] [4, 0] 9.052657127380371 [11.0]
[5.0, 6.0] [51, 0] 9.50953197479248 [11.0]
[0.0, 12.0] [2, 0] 9.31519889831543 [12.0]
[8.0, 4.0] [18, 0] 9.529882431030273 [12.0]
[1.0, 10.0] [2, 0] 9.31519889831543 [11.0]
[6.0, 6.0] [18, 0] 9.529882431030273 [12.0]
[5.0, 7.0] [36, 0] 9.50093936920166 [12.0]
[3.0, 9.0] [51, 0] 9.50953197479248 [12.0]
Testing 20 samples of numbers that sum up to [11 - 12]
Unique outputs: {9.50093936920166, 9.529383659362793, 9.520702362060547, 9

In [121]:
test_some_other_numbers(game3)

[6.0, 4.0] [8, 0] 9.894006729125977 [10.0]
[6.0, 4.0] [7, 0] 9.875419616699219 [10.0]
[9.0, 1.0] [20, 0] 10.464897155761719 [10.0]
[9.0, 1.0] [18, 0] 9.867368698120117 [10.0]
[9.0, 1.0] [21, 0] 9.655567169189453 [10.0]
[1.0, 1.0] [1, 0] 2.8394417762756348 [2.0]
[1.0, 1.0] [1, 0] 2.8394417762756348 [2.0]
[1.0, 1.0] [1, 0] 2.8394417762756348 [2.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[-1.0, 4.0] [1, 0] 2.8394417762756348 [3.0]
[5.0, -2.0] [1, 0] 2.8394417762756348 [3.0]
[-1.0, 0.0] [25, 0] 0.6592649221420288 [-1.0]
[-2.0, -2.0] [25, 0] 0.6592649221420288 [-4.0]
[-9.0, -1.0] [25, 0] 0.6592649221420288 [-10.0]
[8.0, -3.0] [2, 0] 5.553849220275879 [5.0]
[-10.0, -10.0] [25, 0] 0.6592649221420288 [-20.0]
Unique outputs: {0.6592649221420288, 2.8394417762756348, 5.553849220275879, 9.894006729125977, 10.464897155761719, 9.655567169189453, 9.875419616699219, 9.867368698120117}
Accuracy: 0.47058823704719543


In [122]:
test_some_other_numbers(game2)

[6.0, 4.0] [2, 0] 9.31519889831543 [10.0]
[6.0, 4.0] [63, 0] 9.525580406188965 [10.0]
[9.0, 1.0] [2, 0] 9.31519889831543 [10.0]
[9.0, 1.0] [32, 0] 9.501254081726074 [10.0]
[9.0, 1.0] [5, 0] 9.519076347351074 [10.0]
[1.0, 1.0] [17, 0] 1.9181718826293945 [2.0]
[1.0, 1.0] [53, 0] 2.949716091156006 [2.0]
[1.0, 1.0] [17, 0] 1.9181718826293945 [2.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[-1.0, 4.0] [53, 0] 2.949716091156006 [3.0]
[5.0, -2.0] [53, 0] 2.949716091156006 [3.0]
[-1.0, 0.0] [14, 0] 0.6579403281211853 [-1.0]
[-2.0, -2.0] [14, 0] 0.6579403281211853 [-4.0]
[-9.0, -1.0] [14, 0] 0.6579403281211853 [-10.0]
[8.0, -3.0] [9, 0] 5.006842613220215 [5.0]
[-10.0, -10.0] [14, 0] 0.6579403281211853 [-20.0]
Unique outputs: {0.6579403281211853, 1.9181718826293945, 2.949716091156006, 5.006842613220215, 9.31519889831543, 9.519076347351074, 9.525580406188965, 9.501254081726074}
Accuracy: 0.529411792755127


In [126]:
test_some_other_numbers(game)

[6.0, 4.0] [28, 0] 9.217903137207031 [10.0]
[6.0, 4.0] [13, 0] 9.9131498336792 [10.0]
[9.0, 1.0] [13, 0] 9.9131498336792 [10.0]
[9.0, 1.0] [13, 0] 9.9131498336792 [10.0]
[9.0, 1.0] [13, 0] 9.9131498336792 [10.0]
[1.0, 1.0] [21, 0] 1.217471957206726 [2.0]
[1.0, 1.0] [21, 0] 1.217471957206726 [2.0]
[1.0, 1.0] [21, 0] 1.217471957206726 [2.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[-1.0, 4.0] [30, 0] 3.0951414108276367 [3.0]
[5.0, -2.0] [30, 0] 3.0951414108276367 [3.0]
[-1.0, 0.0] [21, 0] 1.217471957206726 [-1.0]
[-2.0, -2.0] [21, 0] 1.217471957206726 [-4.0]
[-9.0, -1.0] [21, 0] 1.217471957206726 [-10.0]
[8.0, -3.0] [9, 0] 5.03515625 [5.0]
[-10.0, -10.0] [21, 0] 1.217471957206726 [-20.0]
Unique outputs: {1.217471957206726, 3.0951414108276367, 5.03515625, 9.217903137207031, 9.9131498336792}
Accuracy: 0.47058823704719543


In [127]:
game4 = load(1500, 1, 64, 64, 64, 7) #game10_14_1_64_64_64_7_1500.tar

Preparing dataset...
Samples in training set: 52
Samples in test set: 14

----------
Building model...
# Initializing model, trainer, and optimizer from game/models/game10_14_1_64_64_64_7_1500.tar
# loading trainer state from game/models/game10_14_1_64_64_64_7_1500.tar
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=64, bias=True)
    )
    (hidden_to_output): Linear(in_features=64, out_features=64, bias=True)
    (embedding): Embedding(64, 7)
    (cells): ModuleList(
      (0): RNNCell(7, 64)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=64, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(7, 64, batch_first=True)
      (embedding): Embedding(64, 7)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
Number of parameters 14664

----------
Start of training... 2021-11-10 21:02:42.492537
{"loss": -0.62

In [133]:
test_all_set(game4, 10)

Testing 66 samples of numbers that sum up to 10
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 2.0] [11, 0] 1.9806139469146729 [2.0]
[0.0, 3.0] [10, 0] 3.0477895736694336 [3.0]
[0.0, 4.0] [41, 0] 3.7039108276367188 [4.0]
[0.0, 5.0] [47, 0] 6.094944000244141 [5.0]
[0.0, 6.0] [47, 0] 6.094944000244141 [6.0]
[0.0, 7.0] [61, 0] 6.4759111404418945 [7.0]
[0.0, 8.0] [43, 0] 8.137287139892578 [8.0]
[0.0, 9.0] [60, 0] 8.249743461608887 [9.0]
[0.0, 10.0] [40, 0] 9.961762428283691 [10.0]
[1.0, 0.0] [38, 0] 0.6050024628639221 [1.0]
[1.0, 1.0] [11, 0] 1.9806139469146729 [2.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 3.0] [50, 0] 3.985212564468384 [4.0]
[1.0, 4.0] [45, 0] 5.259300231933594 [5.0]
[1.0, 5.0] [29, 0] 5.468045234680176 [6.0]
[1.0, 6.0] [34, 0] 7.356264591217041 [7.0]
[1.0, 7.0] [58, 0] 8.322916030883789 [8.0]
[1.0, 8.0] [20, 0] 8.997514724731445 [9.0]
[1.0, 9.0] [24, 0] 9.797828674316406 [10.0]
[2.0, 0.0] [11, 0] 1.9806139469146729

In [134]:
test_above_limit(game4, 11, 12, 20)
test_above_limit(game4, 100, 101, 20)
test_above_limit(game4, 100001, 9999999, 20)


[11.0, 1.0] [24, 0] 9.797828674316406 [12.0]
[2.0, 9.0] [48, 0] 9.935298919677734 [11.0]
[6.0, 5.0] [28, 0] 9.956093788146973 [11.0]
[0.0, 11.0] [28, 0] 9.956093788146973 [11.0]
[1.0, 11.0] [63, 0] 9.941741943359375 [12.0]
[12.0, 0.0] [12, 0] 8.10794448852539 [12.0]
[10.0, 2.0] [55, 0] 9.427413940429688 [12.0]
[9.0, 3.0] [24, 0] 9.797828674316406 [12.0]
[10.0, 1.0] [28, 0] 9.956093788146973 [11.0]
[9.0, 2.0] [48, 0] 9.935298919677734 [11.0]
[7.0, 4.0] [55, 0] 9.427413940429688 [11.0]
[7.0, 5.0] [24, 0] 9.797828674316406 [12.0]
[11.0, 0.0] [48, 0] 9.935298919677734 [11.0]
[5.0, 6.0] [62, 0] 9.953786849975586 [11.0]
[0.0, 12.0] [63, 0] 9.941741943359375 [12.0]
[8.0, 4.0] [24, 0] 9.797828674316406 [12.0]
[1.0, 10.0] [62, 0] 9.953786849975586 [11.0]
[6.0, 6.0] [28, 0] 9.956093788146973 [12.0]
[5.0, 7.0] [24, 0] 9.797828674316406 [12.0]
[3.0, 9.0] [48, 0] 9.935298919677734 [12.0]
Testing 20 samples of numbers that sum up to [11 - 12]
Unique outputs: {8.10794448852539, 9.797828674316406, 9.9

In [137]:
test_some_other_numbers(game4)

[6.0, 4.0] [40, 0] 9.961762428283691 [10.0]
[6.0, 4.0] [48, 0] 9.935298919677734 [10.0]
[9.0, 1.0] [28, 0] 9.956093788146973 [10.0]
[9.0, 1.0] [48, 0] 9.935298919677734 [10.0]
[9.0, 1.0] [62, 0] 9.953786849975586 [10.0]
[1.0, 1.0] [11, 0] 1.9806139469146729 [2.0]
[1.0, 1.0] [11, 0] 1.9806139469146729 [2.0]
[1.0, 1.0] [11, 0] 1.9806139469146729 [2.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[-1.0, 4.0] [10, 0] 3.0477895736694336 [3.0]
[5.0, -2.0] [10, 0] 3.0477895736694336 [3.0]
[-1.0, 0.0] [38, 0] 0.6050024628639221 [-1.0]
[-2.0, -2.0] [38, 0] 0.6050024628639221 [-4.0]
[-9.0, -1.0] [38, 0] 0.6050024628639221 [-10.0]
[8.0, -3.0] [31, 0] 5.870135307312012 [5.0]
[-10.0, -10.0] [38, 0] 0.6050024628639221 [-20.0]
Unique outputs: {0.6050024628639221, 1.9806139469146729, 3.0477895736694336, 5.870135307312012, 9.961762428283691, 9.953786849975586, 9.935298919677734, 9.956093788146973}
Accuracy: 0.6470588445663452


In [184]:
def test_synonymy(game, n, repeat):
    limit = n+1
    for i in range(limit):
        for j in range(limit):
            if(i+j <= n):
                inputs = []
                labels = []
                for p in range(repeat):
                    inputs.append([i, j])
                    labels.append([i+j]) 
                inputs = torch.FloatTensor(inputs)
                labels = torch.FloatTensor(labels)
                prediction = game.game.forward(inputs,labels)
                messages = prediction[1].message.tolist()
                outputs = prediction[1].receiver_output.tolist()
                for q in range(len(outputs)):
                    print("{} {} {} {}".format(inputs[q].tolist(), messages[q], outputs[q][0], labels[q].tolist()))
                print("Unique outputs: {}".format(set([item[0] for item in outputs])))


In [186]:
test_synonymy(game, 10, 10)

[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
[0.0, 0.0] [21, 0] 1.217471957206726 [0.0]
Unique outputs: {1.217471957206726}
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
[0.0, 1.0] [21, 0] 1.217471957206726 [1.0]
Unique outputs: {1.217471957206726}
[0.0, 2.0] [21, 0] 1.217471957206726 [2.0]
[0.0, 2.0] [21, 0] 1.2174

[6.0, 2.0] [22, 0] 8.012168884277344 [8.0]
[6.0, 2.0] [16, 0] 8.039968490600586 [8.0]
[6.0, 2.0] [16, 0] 8.039968490600586 [8.0]
[6.0, 2.0] [16, 0] 8.039968490600586 [8.0]
[6.0, 2.0] [22, 0] 8.012168884277344 [8.0]
[6.0, 2.0] [22, 0] 8.012168884277344 [8.0]
[6.0, 2.0] [22, 0] 8.012168884277344 [8.0]
[6.0, 2.0] [16, 0] 8.039968490600586 [8.0]
[6.0, 2.0] [16, 0] 8.039968490600586 [8.0]
[6.0, 2.0] [16, 0] 8.039968490600586 [8.0]
Unique outputs: {8.012168884277344, 8.039968490600586}
[6.0, 3.0] [2, 0] 9.221352577209473 [9.0]
[6.0, 3.0] [13, 0] 9.9131498336792 [9.0]
[6.0, 3.0] [17, 0] 9.22646427154541 [9.0]
[6.0, 3.0] [13, 0] 9.9131498336792 [9.0]
[6.0, 3.0] [17, 0] 9.22646427154541 [9.0]
[6.0, 3.0] [17, 0] 9.22646427154541 [9.0]
[6.0, 3.0] [28, 0] 9.217903137207031 [9.0]
[6.0, 3.0] [13, 0] 9.9131498336792 [9.0]
[6.0, 3.0] [17, 0] 9.22646427154541 [9.0]
[6.0, 3.0] [26, 0] 9.225600242614746 [9.0]
Unique outputs: {9.221352577209473, 9.217903137207031, 9.225600242614746, 9.9131498336792, 9.226

In [187]:
test_synonymy(game2, 10, 10)

[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
[0.0, 0.0] [14, 0] 0.6579403281211853 [0.0]
Unique outputs: {0.6579403281211853}
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
[0.0, 1.0] [14, 0] 0.6579403281211853 [1.0]
Unique outputs: {0.6579403281211853}
[0.0, 2.0] [17, 0] 1.9181718826293945 [2.0]
[0

[5.0, 1.0] [21, 0] 5.972923755645752 [6.0]
[5.0, 1.0] [42, 0] 5.884256362915039 [6.0]
[5.0, 1.0] [42, 0] 5.884256362915039 [6.0]
[5.0, 1.0] [21, 0] 5.972923755645752 [6.0]
[5.0, 1.0] [44, 0] 5.785057544708252 [6.0]
[5.0, 1.0] [21, 0] 5.972923755645752 [6.0]
[5.0, 1.0] [42, 0] 5.884256362915039 [6.0]
[5.0, 1.0] [42, 0] 5.884256362915039 [6.0]
[5.0, 1.0] [21, 0] 5.972923755645752 [6.0]
[5.0, 1.0] [21, 0] 5.972923755645752 [6.0]
Unique outputs: {5.884256362915039, 5.785057544708252, 5.972923755645752}
[5.0, 2.0] [7, 0] 6.889515399932861 [7.0]
[5.0, 2.0] [57, 0] 6.851171493530273 [7.0]
[5.0, 2.0] [29, 0] 7.1015729904174805 [7.0]
[5.0, 2.0] [34, 0] 6.964462757110596 [7.0]
[5.0, 2.0] [24, 0] 6.985294818878174 [7.0]
[5.0, 2.0] [24, 0] 6.985294818878174 [7.0]
[5.0, 2.0] [29, 0] 7.1015729904174805 [7.0]
[5.0, 2.0] [54, 0] 6.888267993927002 [7.0]
[5.0, 2.0] [24, 0] 6.985294818878174 [7.0]
[5.0, 2.0] [34, 0] 6.964462757110596 [7.0]
Unique outputs: {6.985294818878174, 7.1015729904174805, 6.9644627

In [188]:
test_synonymy(game3, 10, 10)

[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
[0.0, 0.0] [25, 0] 0.6592649221420288 [0.0]
Unique outputs: {0.6592649221420288}
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
[0.0, 1.0] [25, 0] 0.6592649221420288 [1.0]
Unique outputs: {0.6592649221420288}
[0.0, 2.0] [1, 0] 2.8394417762756348 [2.0]
[0.

[7.0, 3.0] [5, 0] 9.90152359008789 [10.0]
[7.0, 3.0] [7, 0] 9.875419616699219 [10.0]
[7.0, 3.0] [20, 0] 10.464897155761719 [10.0]
[7.0, 3.0] [6, 0] 9.84208869934082 [10.0]
[7.0, 3.0] [7, 0] 9.875419616699219 [10.0]
[7.0, 3.0] [7, 0] 9.875419616699219 [10.0]
[7.0, 3.0] [6, 0] 9.84208869934082 [10.0]
[7.0, 3.0] [20, 0] 10.464897155761719 [10.0]
[7.0, 3.0] [20, 0] 10.464897155761719 [10.0]
[7.0, 3.0] [18, 0] 9.867368698120117 [10.0]
Unique outputs: {9.90152359008789, 10.464897155761719, 9.867368698120117, 9.875419616699219, 9.84208869934082}
[8.0, 0.0] [17, 0] 8.081911087036133 [8.0]
[8.0, 0.0] [24, 0] 7.806769371032715 [8.0]
[8.0, 0.0] [4, 0] 8.521757125854492 [8.0]
[8.0, 0.0] [14, 0] 8.346345901489258 [8.0]
[8.0, 0.0] [22, 0] 7.798973560333252 [8.0]
[8.0, 0.0] [22, 0] 7.798973560333252 [8.0]
[8.0, 0.0] [22, 0] 7.798973560333252 [8.0]
[8.0, 0.0] [19, 0] 8.234979629516602 [8.0]
[8.0, 0.0] [0, 0] 6.816980361938477 [8.0]
[8.0, 0.0] [19, 0] 8.234979629516602 [8.0]
Unique outputs: {6.81698036

In [191]:
test_synonymy(game4, 10, 10)

[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
Unique outputs: {0.6050024628639221}
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
[0.0, 1.0] [38, 0] 0.6050024628639221 [1.0]
Unique outputs: {0.6050024628639221}
[0.0, 2.0] [11, 0] 1.9806139469146729 [2.0]
[0

[5.0, 4.0] [17, 0] 9.148031234741211 [9.0]
[5.0, 4.0] [7, 0] 9.29594612121582 [9.0]
[5.0, 4.0] [12, 0] 8.10794448852539 [9.0]
[5.0, 4.0] [35, 0] 9.152091979980469 [9.0]
[5.0, 4.0] [17, 0] 9.148031234741211 [9.0]
[5.0, 4.0] [55, 0] 9.427413940429688 [9.0]
[5.0, 4.0] [7, 0] 9.29594612121582 [9.0]
[5.0, 4.0] [17, 0] 9.148031234741211 [9.0]
[5.0, 4.0] [17, 0] 9.148031234741211 [9.0]
[5.0, 4.0] [14, 0] 9.109984397888184 [9.0]
Unique outputs: {8.10794448852539, 9.148031234741211, 9.427413940429688, 9.29594612121582, 9.152091979980469, 9.109984397888184}
[5.0, 5.0] [28, 0] 9.956093788146973 [10.0]
[5.0, 5.0] [40, 0] 9.961762428283691 [10.0]
[5.0, 5.0] [62, 0] 9.953786849975586 [10.0]
[5.0, 5.0] [48, 0] 9.935298919677734 [10.0]
[5.0, 5.0] [62, 0] 9.953786849975586 [10.0]
[5.0, 5.0] [48, 0] 9.935298919677734 [10.0]
[5.0, 5.0] [62, 0] 9.953786849975586 [10.0]
[5.0, 5.0] [48, 0] 9.935298919677734 [10.0]
[5.0, 5.0] [62, 0] 9.953786849975586 [10.0]
[5.0, 5.0] [48, 0] 9.935298919677734 [10.0]
Unique

In [190]:
test_synonymy(game4, 10, 30)

[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.60500246286

[2.0, 7.0] [20, 0] 8.997514724731445 [9.0]
[2.0, 7.0] [54, 0] 8.877487182617188 [9.0]
[2.0, 7.0] [54, 0] 8.877487182617188 [9.0]
[2.0, 7.0] [12, 0] 8.10794448852539 [9.0]
[2.0, 7.0] [6, 0] 8.740228652954102 [9.0]
[2.0, 7.0] [42, 0] 8.259196281433105 [9.0]
[2.0, 7.0] [54, 0] 8.877487182617188 [9.0]
[2.0, 7.0] [7, 0] 9.29594612121582 [9.0]
[2.0, 7.0] [35, 0] 9.152091979980469 [9.0]
[2.0, 7.0] [14, 0] 9.109984397888184 [9.0]
[2.0, 7.0] [42, 0] 8.259196281433105 [9.0]
[2.0, 7.0] [17, 0] 9.148031234741211 [9.0]
[2.0, 7.0] [48, 0] 9.935298919677734 [9.0]
[2.0, 7.0] [7, 0] 9.29594612121582 [9.0]
[2.0, 7.0] [54, 0] 8.877487182617188 [9.0]
[2.0, 7.0] [7, 0] 9.29594612121582 [9.0]
[2.0, 7.0] [28, 0] 9.956093788146973 [9.0]
[2.0, 7.0] [48, 0] 9.935298919677734 [9.0]
[2.0, 7.0] [12, 0] 8.10794448852539 [9.0]
[2.0, 7.0] [7, 0] 9.29594612121582 [9.0]
[2.0, 7.0] [7, 0] 9.29594612121582 [9.0]
[2.0, 7.0] [7, 0] 9.29594612121582 [9.0]
[2.0, 7.0] [17, 0] 9.148031234741211 [9.0]
[2.0, 7.0] [3, 0] 9.094429

[8.0, 1.0] [48, 0] 9.935298919677734 [9.0]
[8.0, 1.0] [55, 0] 9.427413940429688 [9.0]
[8.0, 1.0] [28, 0] 9.956093788146973 [9.0]
[8.0, 1.0] [17, 0] 9.148031234741211 [9.0]
[8.0, 1.0] [57, 0] 8.850218772888184 [9.0]
[8.0, 1.0] [55, 0] 9.427413940429688 [9.0]
[8.0, 1.0] [63, 0] 9.941741943359375 [9.0]
[8.0, 1.0] [54, 0] 8.877487182617188 [9.0]
[8.0, 1.0] [35, 0] 9.152091979980469 [9.0]
[8.0, 1.0] [62, 0] 9.953786849975586 [9.0]
[8.0, 1.0] [7, 0] 9.29594612121582 [9.0]
[8.0, 1.0] [48, 0] 9.935298919677734 [9.0]
[8.0, 1.0] [24, 0] 9.797828674316406 [9.0]
[8.0, 1.0] [17, 0] 9.148031234741211 [9.0]
[8.0, 1.0] [17, 0] 9.148031234741211 [9.0]
[8.0, 1.0] [57, 0] 8.850218772888184 [9.0]
[8.0, 1.0] [35, 0] 9.152091979980469 [9.0]
[8.0, 1.0] [27, 0] 9.479999542236328 [9.0]
[8.0, 1.0] [24, 0] 9.797828674316406 [9.0]
[8.0, 1.0] [48, 0] 9.935298919677734 [9.0]
[8.0, 1.0] [28, 0] 9.956093788146973 [9.0]
[8.0, 1.0] [35, 0] 9.152091979980469 [9.0]
[8.0, 1.0] [7, 0] 9.29594612121582 [9.0]
[8.0, 1.0] [14,

In [192]:
test_synonymy(game4, 10, 50)

[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.60500246286

[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.9806139469146729 [2.0]
[2.0, 0.0] [11, 0] 1.98061394691

[4.0, 1.0] [31, 0] 5.870135307312012 [5.0]
[4.0, 1.0] [29, 0] 5.468045234680176 [5.0]
[4.0, 1.0] [45, 0] 5.259300231933594 [5.0]
[4.0, 1.0] [47, 0] 6.094944000244141 [5.0]
[4.0, 1.0] [0, 0] 5.242588996887207 [5.0]
[4.0, 1.0] [45, 0] 5.259300231933594 [5.0]
[4.0, 1.0] [0, 0] 5.242588996887207 [5.0]
[4.0, 1.0] [22, 0] 5.7301106452941895 [5.0]
[4.0, 1.0] [59, 0] 5.25497579574585 [5.0]
[4.0, 1.0] [0, 0] 5.242588996887207 [5.0]
[4.0, 1.0] [45, 0] 5.259300231933594 [5.0]
[4.0, 1.0] [0, 0] 5.242588996887207 [5.0]
[4.0, 1.0] [59, 0] 5.25497579574585 [5.0]
[4.0, 1.0] [22, 0] 5.7301106452941895 [5.0]
[4.0, 1.0] [45, 0] 5.259300231933594 [5.0]
[4.0, 1.0] [22, 0] 5.7301106452941895 [5.0]
[4.0, 1.0] [31, 0] 5.870135307312012 [5.0]
[4.0, 1.0] [41, 0] 3.7039108276367188 [5.0]
[4.0, 1.0] [22, 0] 5.7301106452941895 [5.0]
[4.0, 1.0] [0, 0] 5.242588996887207 [5.0]
[4.0, 1.0] [45, 0] 5.259300231933594 [5.0]
[4.0, 1.0] [22, 0] 5.7301106452941895 [5.0]
[4.0, 1.0] [22, 0] 5.7301106452941895 [5.0]
[4.0, 1.0] 

[6.0, 2.0] [23, 0] 8.084556579589844 [8.0]
[6.0, 2.0] [23, 0] 8.084556579589844 [8.0]
[6.0, 2.0] [60, 0] 8.249743461608887 [8.0]
[6.0, 2.0] [12, 0] 8.10794448852539 [8.0]
[6.0, 2.0] [51, 0] 7.999384880065918 [8.0]
[6.0, 2.0] [42, 0] 8.259196281433105 [8.0]
[6.0, 2.0] [42, 0] 8.259196281433105 [8.0]
[6.0, 2.0] [12, 0] 8.10794448852539 [8.0]
[6.0, 2.0] [43, 0] 8.137287139892578 [8.0]
[6.0, 2.0] [42, 0] 8.259196281433105 [8.0]
[6.0, 2.0] [42, 0] 8.259196281433105 [8.0]
[6.0, 2.0] [37, 0] 8.130910873413086 [8.0]
[6.0, 2.0] [58, 0] 8.322916030883789 [8.0]
[6.0, 2.0] [9, 0] 7.370023727416992 [8.0]
[6.0, 2.0] [7, 0] 9.29594612121582 [8.0]
[6.0, 2.0] [58, 0] 8.322916030883789 [8.0]
[6.0, 2.0] [58, 0] 8.322916030883789 [8.0]
[6.0, 2.0] [12, 0] 8.10794448852539 [8.0]
[6.0, 2.0] [12, 0] 8.10794448852539 [8.0]
[6.0, 2.0] [12, 0] 8.10794448852539 [8.0]
[6.0, 2.0] [42, 0] 8.259196281433105 [8.0]
[6.0, 2.0] [60, 0] 8.249743461608887 [8.0]
[6.0, 2.0] [58, 0] 8.322916030883789 [8.0]
[6.0, 2.0] [58, 0] 

In [193]:
test_synonymy(game4, 10, 100)

[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.6050024628639221 [0.0]
[0.0, 0.0] [38, 0] 0.60500246286

[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [41, 0] 3.7039108276367188 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.0477895736694336 [3.0]
[1.0, 2.0] [10, 0] 3.04778957366

[1.0, 8.0] [35, 0] 9.152091979980469 [9.0]
[1.0, 8.0] [7, 0] 9.29594612121582 [9.0]
[1.0, 8.0] [13, 0] 9.271989822387695 [9.0]
[1.0, 8.0] [12, 0] 8.10794448852539 [9.0]
[1.0, 8.0] [35, 0] 9.152091979980469 [9.0]
[1.0, 8.0] [17, 0] 9.148031234741211 [9.0]
[1.0, 8.0] [7, 0] 9.29594612121582 [9.0]
[1.0, 8.0] [35, 0] 9.152091979980469 [9.0]
[1.0, 8.0] [3, 0] 9.094429016113281 [9.0]
[1.0, 8.0] [17, 0] 9.148031234741211 [9.0]
[1.0, 8.0] [54, 0] 8.877487182617188 [9.0]
[1.0, 8.0] [3, 0] 9.094429016113281 [9.0]
[1.0, 8.0] [35, 0] 9.152091979980469 [9.0]
[1.0, 8.0] [6, 0] 8.740228652954102 [9.0]
[1.0, 8.0] [7, 0] 9.29594612121582 [9.0]
[1.0, 8.0] [54, 0] 8.877487182617188 [9.0]
[1.0, 8.0] [54, 0] 8.877487182617188 [9.0]
[1.0, 8.0] [28, 0] 9.956093788146973 [9.0]
[1.0, 8.0] [16, 0] 7.680649757385254 [9.0]
[1.0, 8.0] [7, 0] 9.29594612121582 [9.0]
[1.0, 8.0] [42, 0] 8.259196281433105 [9.0]
[1.0, 8.0] [28, 0] 9.956093788146973 [9.0]
[1.0, 8.0] [12, 0] 8.10794448852539 [9.0]
[1.0, 8.0] [3, 0] 9.0944

[2.0, 6.0] [54, 0] 8.877487182617188 [8.0]
[2.0, 6.0] [42, 0] 8.259196281433105 [8.0]
[2.0, 6.0] [58, 0] 8.322916030883789 [8.0]
[2.0, 6.0] [58, 0] 8.322916030883789 [8.0]
[2.0, 6.0] [39, 0] 8.254074096679688 [8.0]
[2.0, 6.0] [60, 0] 8.249743461608887 [8.0]
[2.0, 6.0] [47, 0] 6.094944000244141 [8.0]
[2.0, 6.0] [42, 0] 8.259196281433105 [8.0]
[2.0, 6.0] [43, 0] 8.137287139892578 [8.0]
[2.0, 6.0] [42, 0] 8.259196281433105 [8.0]
[2.0, 6.0] [37, 0] 8.130910873413086 [8.0]
[2.0, 6.0] [12, 0] 8.10794448852539 [8.0]
[2.0, 6.0] [42, 0] 8.259196281433105 [8.0]
[2.0, 6.0] [37, 0] 8.130910873413086 [8.0]
[2.0, 6.0] [39, 0] 8.254074096679688 [8.0]
[2.0, 6.0] [58, 0] 8.322916030883789 [8.0]
[2.0, 6.0] [30, 0] 7.502965927124023 [8.0]
[2.0, 6.0] [25, 0] 7.937762260437012 [8.0]
[2.0, 6.0] [58, 0] 8.322916030883789 [8.0]
[2.0, 6.0] [42, 0] 8.259196281433105 [8.0]
[2.0, 6.0] [12, 0] 8.10794448852539 [8.0]
[2.0, 6.0] [60, 0] 8.249743461608887 [8.0]
[2.0, 6.0] [42, 0] 8.259196281433105 [8.0]
[2.0, 6.0] [2

[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [34, 0] 7.356264591217041 [7.0]
[3.0, 4.0] [30, 0] 7.502965927124023 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [30, 0] 7.502965927124023 [7.0]
[3.0, 4.0] [26, 0] 7.234463214874268 [7.0]
[3.0, 4.0] [47, 0] 6.094944000244141 [7.0]
[3.0, 4.0] [34, 0] 7.356264591217041 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [19, 0] 7.1300458908081055 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [56, 0] 7.442014694213867 [7.0]
[3.0, 4.0] [34, 0] 7.356264591217041 [7.0]
[3.0, 4.0] [37, 0] 8.130910873413086 [7.0]
[3.0, 4.0] [19, 0] 7.1300458908081055 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [9, 0] 7.370023727416992 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0] [44, 0] 7.253515243530273 [7.0]
[3.0, 4.0]

[4.0, 2.0] [22, 0] 5.7301106452941895 [6.0]
[4.0, 2.0] [31, 0] 5.870135307312012 [6.0]
[4.0, 2.0] [31, 0] 5.870135307312012 [6.0]
[4.0, 2.0] [22, 0] 5.7301106452941895 [6.0]
[4.0, 2.0] [15, 0] 5.933892250061035 [6.0]
[4.0, 2.0] [29, 0] 5.468045234680176 [6.0]
[4.0, 2.0] [47, 0] 6.094944000244141 [6.0]
[4.0, 2.0] [45, 0] 5.259300231933594 [6.0]
[4.0, 2.0] [22, 0] 5.7301106452941895 [6.0]
[4.0, 2.0] [15, 0] 5.933892250061035 [6.0]
[4.0, 2.0] [22, 0] 5.7301106452941895 [6.0]
[4.0, 2.0] [31, 0] 5.870135307312012 [6.0]
[4.0, 2.0] [21, 0] 7.669336318969727 [6.0]
[4.0, 2.0] [15, 0] 5.933892250061035 [6.0]
[4.0, 2.0] [47, 0] 6.094944000244141 [6.0]
[4.0, 2.0] [44, 0] 7.253515243530273 [6.0]
[4.0, 2.0] [31, 0] 5.870135307312012 [6.0]
[4.0, 2.0] [22, 0] 5.7301106452941895 [6.0]
[4.0, 2.0] [45, 0] 5.259300231933594 [6.0]
[4.0, 2.0] [31, 0] 5.870135307312012 [6.0]
[4.0, 2.0] [31, 0] 5.870135307312012 [6.0]
[4.0, 2.0] [47, 0] 6.094944000244141 [6.0]
[4.0, 2.0] [45, 0] 5.259300231933594 [6.0]
[4.0, 

[5.0, 0.0] [22, 0] 5.7301106452941895 [5.0]
[5.0, 0.0] [29, 0] 5.468045234680176 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [59, 0] 5.25497579574585 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [0, 0] 5.242588996887207 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [29, 0] 5.468045234680176 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [0, 0] 5.242588996887207 [5.0]
[5.0, 0.0] [41, 0] 3.7039108276367188 [5.0]
[5.0, 0.0] [0, 0] 5.242588996887207 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [0, 0] 5.242588996887207 [5.0]
[5.0, 0.0] [0, 0] 5.242588996887207 [5.0]
[5.0, 0.0] [36, 0] 6.203551292419434 [5.0]
[5.0, 0.0] [0, 0] 5.242588996887207 [5.0]
[5.0, 0.0] [0, 0] 5.242588996887207 [5.0]
[5.0, 0.0] [59, 0] 5.25497579574585 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [22, 0] 5.7301106452941895 [5.0]
[5.0, 0.0] [45, 0] 5.259300231933594 [5.0]
[5.0, 0.0] [0, 0]

[6.0, 4.0] [63, 0] 9.941741943359375 [10.0]
[6.0, 4.0] [40, 0] 9.961762428283691 [10.0]
[6.0, 4.0] [24, 0] 9.797828674316406 [10.0]
[6.0, 4.0] [20, 0] 8.997514724731445 [10.0]
[6.0, 4.0] [62, 0] 9.953786849975586 [10.0]
[6.0, 4.0] [28, 0] 9.956093788146973 [10.0]
[6.0, 4.0] [24, 0] 9.797828674316406 [10.0]
[6.0, 4.0] [24, 0] 9.797828674316406 [10.0]
[6.0, 4.0] [62, 0] 9.953786849975586 [10.0]
[6.0, 4.0] [62, 0] 9.953786849975586 [10.0]
[6.0, 4.0] [48, 0] 9.935298919677734 [10.0]
[6.0, 4.0] [28, 0] 9.956093788146973 [10.0]
[6.0, 4.0] [63, 0] 9.941741943359375 [10.0]
[6.0, 4.0] [27, 0] 9.479999542236328 [10.0]
[6.0, 4.0] [40, 0] 9.961762428283691 [10.0]
[6.0, 4.0] [48, 0] 9.935298919677734 [10.0]
[6.0, 4.0] [40, 0] 9.961762428283691 [10.0]
[6.0, 4.0] [62, 0] 9.953786849975586 [10.0]
[6.0, 4.0] [48, 0] 9.935298919677734 [10.0]
[6.0, 4.0] [48, 0] 9.935298919677734 [10.0]
[6.0, 4.0] [24, 0] 9.797828674316406 [10.0]
[6.0, 4.0] [62, 0] 9.953786849975586 [10.0]
[6.0, 4.0] [28, 0] 9.95609378814

[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [13, 0] 9.271989822387695 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [28, 0] 9.956093788146973 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [62, 0] 9.953786849975586 [10.0]
[8.0, 2.0] [63, 0] 9.941741943359375 [10.0]
[8.0, 2.0] [48, 0] 9.935298919677734 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [48, 0] 9.935298919677734 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [28, 0] 9.956093788146973 [10.0]
[8.0, 2.0] [40, 0] 9.961762428283691 [10.0]
[8.0, 2.0] [48, 0] 9.935298919677734 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [28, 0] 9.956093788146973 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [24, 0] 9.797828674316406 [10.0]
[8.0, 2.0] [28, 0] 9.956093788146973 [10.0]
[8.0, 2.0] [20, 0] 8.99751472473

In [211]:
def load_len(last_save, length, n, vocab, sender, receiver, embedding):
    epochs = last_save + 1
    save_name = "game/models/game10_15_{}_{}_{}_{}_{}_{}".format(n, vocab, sender, receiver, embedding, length)
    params = [
        '--batch_size=1',
        '--n_epochs={}'.format(epochs),
        '--vocab_size={}'.format(vocab),
        '--sender_hidden={}'.format(sender),
        '--receiver_hidden={}'.format(receiver),
        '--lr=0.001',
        '--max_len={}'.format(length),
        '--sender_entropy=0.5',
        '--sender_embedding={}'.format(embedding),
        '--receiver_embedding={}'.format(embedding),
        '--validation_freq=10',
        '--train_data=game/data/train10.txt',
        '--validation_data=game/data/validation10.txt',
        '--load_from_checkpoint={}_{}.tar'.format(save_name, last_save)
    ]
    game = Game(params)
    game.play()
    game.save_checkpoint(epochs, save_name)
    return game

In [212]:
game5 = load_len(1500, 2, 1, 64, 64, 64, 7) #game10_15_1_64_64_64_7_2_1500.tar

Preparing dataset...
Samples in training set: 52
Samples in test set: 14

----------
Building model...
# Initializing model, trainer, and optimizer from game/models/game10_15_1_64_64_64_7_2_1500.tar
# loading trainer state from game/models/game10_15_1_64_64_64_7_2_1500.tar
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=64, bias=True)
    )
    (hidden_to_output): Linear(in_features=64, out_features=64, bias=True)
    (embedding): Embedding(64, 7)
    (cells): ModuleList(
      (0): RNNCell(7, 64)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=64, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(7, 64, batch_first=True)
      (embedding): Embedding(64, 7)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
Number of parameters 14664

----------
Start of training... 2021-11-10 21:39:39.372522
{"loss": -

In [213]:
test_all_set(game5, 10)

Testing 66 samples of numbers that sum up to 10
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 1.0] [47, 9, 0] 0.734203040599823 [1.0]
[0.0, 2.0] [9, 9, 0] 2.595468044281006 [2.0]
[0.0, 3.0] [9, 9, 0] 2.595468044281006 [3.0]
[0.0, 4.0] [24, 9, 0] 4.025713920593262 [4.0]
[0.0, 5.0] [20, 47, 0] 5.680097579956055 [5.0]
[0.0, 6.0] [28, 47, 0] 5.666585445404053 [6.0]
[0.0, 7.0] [25, 9, 0] 6.974668979644775 [7.0]
[0.0, 8.0] [12, 24, 0] 8.052736282348633 [8.0]
[0.0, 9.0] [58, 52, 0] 9.013500213623047 [9.0]
[0.0, 10.0] [12, 36, 0] 10.001254081726074 [10.0]
[1.0, 0.0] [47, 9, 0] 0.734203040599823 [1.0]
[1.0, 1.0] [9, 9, 0] 2.595468044281006 [2.0]
[1.0, 2.0] [9, 9, 0] 2.595468044281006 [3.0]
[1.0, 3.0] [57, 47, 0] 3.8633899688720703 [4.0]
[1.0, 4.0] [5, 47, 0] 5.669319152832031 [5.0]
[1.0, 5.0] [34, 9, 0] 6.980932235717773 [6.0]
[1.0, 6.0] [59, 9, 0] 6.988523483276367 [7.0]
[1.0, 7.0] [32, 42, 0] 8.022241592407227 [8.0]
[1.0, 8.0] [20, 8, 0] 9.00475788116455 [9.0]
[1.0, 9.0] [53, 53, 0] 10.

In [214]:
test_above_limit(game5, 11, 12, 20)
test_above_limit(game5, 100, 101, 20)
test_above_limit(game5, 100001, 9999999, 20)


[11.0, 1.0] [4, 18, 0] 10.010160446166992 [12.0]
[2.0, 9.0] [12, 44, 0] 10.002400398254395 [11.0]
[6.0, 5.0] [10, 4, 0] 10.00497055053711 [11.0]
[0.0, 11.0] [58, 3, 0] 10.010116577148438 [11.0]
[1.0, 11.0] [54, 3, 0] 10.011749267578125 [12.0]
[12.0, 0.0] [12, 18, 0] 10.008214950561523 [12.0]
[10.0, 2.0] [22, 43, 0] 10.028193473815918 [12.0]
[9.0, 3.0] [37, 58, 0] 10.019058227539062 [12.0]
[10.0, 1.0] [18, 58, 0] 10.007879257202148 [11.0]
[9.0, 2.0] [32, 18, 0] 10.002880096435547 [11.0]
[7.0, 4.0] [58, 44, 0] 9.995758056640625 [11.0]
[7.0, 5.0] [19, 43, 0] 10.014884948730469 [12.0]
[11.0, 0.0] [32, 12, 0] 10.004135131835938 [11.0]
[5.0, 6.0] [21, 25, 0] 10.00020980834961 [11.0]
[0.0, 12.0] [60, 43, 0] 10.009459495544434 [12.0]
[8.0, 4.0] [28, 3, 0] 10.021807670593262 [12.0]
[1.0, 10.0] [35, 5, 0] 9.994869232177734 [11.0]
[6.0, 6.0] [45, 61, 0] 10.005809783935547 [12.0]
[5.0, 7.0] [60, 61, 0] 10.001380920410156 [12.0]
[3.0, 9.0] [53, 45, 0] 10.00338363647461 [12.0]
Testing 20 samples of 

In [215]:
test_some_other_numbers(game5)

[6.0, 4.0] [44, 44, 0] 9.993438720703125 [10.0]
[6.0, 4.0] [45, 43, 0] 10.018306732177734 [10.0]
[9.0, 1.0] [18, 35, 0] 10.008374214172363 [10.0]
[9.0, 1.0] [20, 18, 0] 10.005900382995605 [10.0]
[9.0, 1.0] [40, 6, 0] 10.001766204833984 [10.0]
[1.0, 1.0] [9, 9, 0] 2.595468044281006 [2.0]
[1.0, 1.0] [9, 9, 0] 2.595468044281006 [2.0]
[1.0, 1.0] [9, 9, 0] 2.595468044281006 [2.0]
[0.0, 1.0] [47, 9, 0] 0.734203040599823 [1.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[-1.0, 4.0] [9, 9, 0] 2.595468044281006 [3.0]
[5.0, -2.0] [9, 9, 0] 2.595468044281006 [3.0]
[-1.0, 0.0] [47, 9, 0] 0.734203040599823 [-1.0]
[-2.0, -2.0] [47, 9, 0] 0.734203040599823 [-4.0]
[-9.0, -1.0] [47, 9, 0] 0.734203040599823 [-10.0]
[8.0, -3.0] [59, 47, 0] 5.6806559562683105 [5.0]
[-10.0, -10.0] [47, 9, 0] 0.734203040599823 [-20.0]
Unique outputs: {0.734203040599823, 2.595468044281006, 5.6806559562683105, 9.993438720703125, 10.005900382995605, 10.018306732177734, 10.008374214172363, 10.001766204833984}
Accuracy: 0.4705

In [216]:
test_synonymy(game5, 10, 100)

[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 47, 0] 0.7238280177116394 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 47, 0] 0.7238280177116394 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203040599823 [0.0]
[0.0, 0.0] [47, 9, 0] 0.734203

[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [24, 47, 0] 4.027108192443848 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [24, 9, 0] 4.025713920593262 [4.0]
[1.0, 3.0] [24, 47, 0] 4.027108192443848 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [57, 9, 0] 3.898606300354004 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [24, 9, 0] 4.025713920593262 [4.0]
[1.0, 3.0] [24, 47, 0] 4.027108192443848 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [24, 47, 0] 4.027108192443848 [4.0]
[1.0, 3.0] [57, 9, 0] 3.898606300354004 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [24, 47, 0] 4.027108192443848 [4.0]
[1.0, 3.0] [24, 9, 0] 4.025713920593262 [4.0]
[1.0, 3.0] [42, 47, 0] 4.0564775466918945 [4.0]
[1.0, 3.0] [24, 9, 0] 4.025713920593262 [4.0]
[1.0, 3.0

[2.0, 3.0] [38, 47, 0] 5.071222305297852 [5.0]
[2.0, 3.0] [58, 47, 0] 5.67434549331665 [5.0]
[2.0, 3.0] [39, 47, 0] 5.681970119476318 [5.0]
[2.0, 3.0] [44, 47, 0] 5.674214839935303 [5.0]
[2.0, 3.0] [42, 47, 0] 4.0564775466918945 [5.0]
[2.0, 3.0] [14, 47, 0] 5.657970905303955 [5.0]
[2.0, 3.0] [42, 47, 0] 4.0564775466918945 [5.0]
[2.0, 3.0] [60, 47, 0] 5.6849894523620605 [5.0]
[2.0, 3.0] [7, 47, 0] 5.6484527587890625 [5.0]
[2.0, 3.0] [13, 47, 0] 5.667064189910889 [5.0]
[2.0, 3.0] [4, 47, 0] 5.650085926055908 [5.0]
[2.0, 3.0] [38, 47, 0] 5.071222305297852 [5.0]
[2.0, 3.0] [7, 47, 0] 5.6484527587890625 [5.0]
[2.0, 3.0] [16, 47, 0] 5.652198791503906 [5.0]
[2.0, 3.0] [7, 47, 0] 5.6484527587890625 [5.0]
[2.0, 3.0] [41, 47, 0] 5.666985511779785 [5.0]
[2.0, 3.0] [38, 47, 0] 5.071222305297852 [5.0]
[2.0, 3.0] [58, 47, 0] 5.67434549331665 [5.0]
[2.0, 3.0] [56, 47, 0] 5.66540002822876 [5.0]
[2.0, 3.0] [13, 47, 0] 5.667064189910889 [5.0]
[2.0, 3.0] [18, 47, 0] 5.67549467086792 [5.0]
[2.0, 3.0] [13,

[3.0, 2.0] [14, 47, 0] 5.657970905303955 [5.0]
[3.0, 2.0] [7, 47, 0] 5.6484527587890625 [5.0]
[3.0, 2.0] [42, 47, 0] 4.0564775466918945 [5.0]
[3.0, 2.0] [57, 9, 0] 3.898606300354004 [5.0]
[3.0, 2.0] [38, 47, 0] 5.071222305297852 [5.0]
[3.0, 2.0] [38, 47, 0] 5.071222305297852 [5.0]
[3.0, 2.0] [55, 47, 0] 5.065963268280029 [5.0]
[3.0, 2.0] [17, 47, 0] 5.674052715301514 [5.0]
[3.0, 2.0] [16, 47, 0] 5.652198791503906 [5.0]
[3.0, 2.0] [60, 47, 0] 5.6849894523620605 [5.0]
[3.0, 2.0] [38, 47, 0] 5.071222305297852 [5.0]
[3.0, 2.0] [57, 47, 0] 3.8633899688720703 [5.0]
[3.0, 2.0] [8, 47, 0] 5.267160892486572 [5.0]
[3.0, 2.0] [37, 47, 0] 5.68383264541626 [5.0]
[3.0, 2.0] [42, 47, 0] 4.0564775466918945 [5.0]
[3.0, 2.0] [38, 47, 0] 5.071222305297852 [5.0]
[3.0, 2.0] [24, 47, 0] 4.027108192443848 [5.0]
[3.0, 2.0] [52, 47, 0] 5.083771705627441 [5.0]
[3.0, 2.0] [7, 47, 0] 5.6484527587890625 [5.0]
[3.0, 2.0] [21, 47, 0] 5.656863689422607 [5.0]
[3.0, 2.0] [19, 47, 0] 5.660178184509277 [5.0]
[3.0, 2.0] [

[4.0, 5.0] [59, 51, 0] 9.00446891784668 [9.0]
[4.0, 5.0] [18, 63, 0] 9.025144577026367 [9.0]
[4.0, 5.0] [58, 52, 0] 9.013500213623047 [9.0]
[4.0, 5.0] [56, 8, 0] 9.017312049865723 [9.0]
[4.0, 5.0] [18, 8, 0] 8.994344711303711 [9.0]
[4.0, 5.0] [44, 52, 0] 9.001350402832031 [9.0]
[4.0, 5.0] [53, 51, 0] 9.00600814819336 [9.0]
[4.0, 5.0] [10, 51, 0] 9.004915237426758 [9.0]
[4.0, 5.0] [49, 33, 0] 9.02308177947998 [9.0]
[4.0, 5.0] [60, 0, 0] 8.811243057250977 [9.0]
[4.0, 5.0] [18, 8, 0] 8.994344711303711 [9.0]
[4.0, 5.0] [60, 0, 0] 8.811243057250977 [9.0]
[4.0, 5.0] [25, 63, 0] 9.033021926879883 [9.0]
[4.0, 5.0] [43, 51, 0] 9.009393692016602 [9.0]
[4.0, 5.0] [58, 0, 0] 8.858024597167969 [9.0]
[4.0, 5.0] [44, 52, 0] 9.001350402832031 [9.0]
[4.0, 5.0] [58, 0, 0] 8.858024597167969 [9.0]
[4.0, 5.0] [31, 8, 0] 8.993094444274902 [9.0]
[4.0, 5.0] [17, 33, 0] 9.01873779296875 [9.0]
[4.0, 5.0] [59, 52, 0] 9.00473403930664 [9.0]
[4.0, 5.0] [2, 51, 0] 9.001518249511719 [9.0]
[4.0, 5.0] [15, 52, 0] 9.00

[6.0, 1.0] [35, 9, 0] 6.961080074310303 [7.0]
[6.0, 1.0] [20, 9, 0] 6.999823093414307 [7.0]
[6.0, 1.0] [17, 9, 0] 6.979401111602783 [7.0]
[6.0, 1.0] [3, 9, 0] 6.9790449142456055 [7.0]
[6.0, 1.0] [20, 9, 0] 6.999823093414307 [7.0]
[6.0, 1.0] [58, 9, 0] 6.974497318267822 [7.0]
[6.0, 1.0] [58, 9, 0] 6.974497318267822 [7.0]
[6.0, 1.0] [20, 9, 0] 6.999823093414307 [7.0]
[6.0, 1.0] [38, 9, 0] 6.9071044921875 [7.0]
[6.0, 1.0] [18, 9, 0] 6.985917568206787 [7.0]
[6.0, 1.0] [14, 9, 0] 6.970404148101807 [7.0]
[6.0, 1.0] [14, 9, 0] 6.970404148101807 [7.0]
[6.0, 1.0] [13, 9, 0] 6.972742557525635 [7.0]
[6.0, 1.0] [46, 9, 0] 6.970003604888916 [7.0]
[6.0, 1.0] [20, 9, 0] 6.999823093414307 [7.0]
[6.0, 1.0] [44, 9, 0] 6.969730854034424 [7.0]
[6.0, 1.0] [45, 9, 0] 7.0073161125183105 [7.0]
[6.0, 1.0] [18, 9, 0] 6.985917568206787 [7.0]
[6.0, 1.0] [18, 9, 0] 6.985917568206787 [7.0]
[6.0, 1.0] [14, 9, 0] 6.970404148101807 [7.0]
[6.0, 1.0] [10, 9, 0] 6.987484931945801 [7.0]
[6.0, 1.0] [41, 9, 0] 6.96752691268

[7.0, 3.0] [44, 53, 0] 10.00450611114502 [10.0]
[7.0, 3.0] [59, 28, 0] 9.999465942382812 [10.0]
[7.0, 3.0] [2, 40, 0] 10.01784896850586 [10.0]
[7.0, 3.0] [25, 12, 0] 10.007135391235352 [10.0]
[7.0, 3.0] [2, 32, 0] 10.011482238769531 [10.0]
[7.0, 3.0] [12, 58, 0] 10.017913818359375 [10.0]
[7.0, 3.0] [18, 60, 0] 9.987415313720703 [10.0]
[7.0, 3.0] [28, 43, 0] 10.02386474609375 [10.0]
[7.0, 3.0] [18, 60, 0] 9.987415313720703 [10.0]
[7.0, 3.0] [21, 21, 0] 9.987876892089844 [10.0]
[7.0, 3.0] [20, 4, 0] 10.0113525390625 [10.0]
[7.0, 3.0] [53, 12, 0] 10.003518104553223 [10.0]
[7.0, 3.0] [43, 19, 0] 9.98780345916748 [10.0]
[7.0, 3.0] [7, 14, 0] 9.996572494506836 [10.0]
[7.0, 3.0] [6, 22, 0] 9.509053230285645 [10.0]
[7.0, 3.0] [61, 19, 0] 10.019192695617676 [10.0]
[7.0, 3.0] [53, 6, 0] 9.99863052368164 [10.0]
[7.0, 3.0] [21, 40, 0] 10.003767013549805 [10.0]
[7.0, 3.0] [2, 18, 0] 10.00570011138916 [10.0]
[7.0, 3.0] [16, 32, 0] 10.011524200439453 [10.0]
[7.0, 3.0] [18, 21, 0] 9.995199203491211 [1